In [110]:
import os
import logging
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, when, desc, avg, min, udf, percent_rank,countDistinct,stddev,lit,create_map,coalesce
)
from pyspark.sql.types import ( StringType, FloatType
)
from pyspark.ml.feature import (
    VectorAssembler, StandardScaler, StringIndexer, Bucketizer
)
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql.functions import concat_ws
from pyspark.ml.clustering import KMeans
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import ClusteringEvaluator, RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.window import Window
# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("modelisation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Paramètres globaux
DATA_DIR = "./data/processed/parquet"
MODELS_DIR = "./models"
RESULTS_DIR = "./results"

# Création des répertoires de sortie s'ils n'existent pas
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Création de la session Spark
def create_spark_session():
    """Crée et retourne une session Spark configurée avec optimisations"""
    return SparkSession.builder \
        .appName("E-commerce Models") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.memory.fraction", "0.8") \
        .config("spark.memory.storageFraction", "0.3") \
        .config("spark.executor.memoryOverhead", "1g") \
        .config("spark.sql.session.timeZone", "UTC") \
        .config("spark.sql.shuffle.partitions", "24") \
        .config("spark.default.parallelism", "8") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
        .config("spark.python.worker.reuse", "true") \
        .config("spark.python.worker.memory", "2g") \
        .config("spark.network.timeout", "800s") \
        .config("spark.locality.wait", "10s") \
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "1000") \
        .master("local[8]") \
        .getOrCreate()

In [111]:
def find_latest_file(directory, pattern):
    """Trouve le fichier le plus récent dans le répertoire correspondant au pattern"""
    files = [f for f in os.listdir(directory) if pattern in f]
    if not files:
        return None
    
    latest_file = None
    latest_time = 0
    
    for file in files:
        file_path = os.path.join(directory, file)
        file_time = os.path.getmtime(file_path)
        if file_time > latest_time:
            latest_time = file_time
            latest_file = file
            
    return latest_file

In [112]:
def prepare_rfm_segmentation(user_df):
    """Prépare les données pour la segmentation RFM avec améliorations"""
    logger.info("Préparation des données pour segmentation RFM")
    
    # Filtrer les utilisateurs avec au moins une interaction et mettre en cache
    rfm_df = user_df.filter(col("nb_events") > 0).cache()
    
    # Convertir les valeurs manquantes/nulles en 0 pour les métriques RFM
    rfm_df = rfm_df.fillna({
        "recency": 30,  # Valeur max si jamais vu
        "frequency": 0,  # Pas d'achats
        "monetary": 0    # Pas de dépenses
    })
    
    # Créer des buckets pour les métriques RFM - version simplifiée pour éviter les erreurs
    # Utilisation de quantiles fixes pour plus de stabilité
    
    # Récence (inversée: plus petit = meilleur)
    recency_splits = [0, 10, 20, 30, float('inf')]
    
    # Fréquence et Monétaire - calcul dynamique sécurisé
    freq_stats = rfm_df.select(
        avg("frequency").alias("avg_freq"),
        stddev("frequency").alias("std_freq")
    ).collect()[0]
    
    # Si pas de variance, utiliser des splits fixes
    if freq_stats["std_freq"] is None or freq_stats["std_freq"] == 0:
        frequency_splits = [0, 0.5, 1.5, 2.5, float('inf')]
    else:
        avg_freq = freq_stats["avg_freq"] or 0
        frequency_splits = [0, avg_freq * 0.5, avg_freq, avg_freq * 1.5, float('inf')]
    
    # Même logique pour monetary
    monetary_stats = rfm_df.select(
        avg("monetary").alias("avg_monetary"),
        stddev("monetary").alias("std_monetary")
    ).collect()[0]
    
    if monetary_stats["std_monetary"] is None or monetary_stats["std_monetary"] == 0:
        monetary_splits = [0, 100, 500, 1000, float('inf')]
    else:
        avg_monetary = monetary_stats["avg_monetary"] or 0
        monetary_splits = [0, avg_monetary * 0.5, avg_monetary, avg_monetary * 1.5, float('inf')]
    
    logger.info(f"Splits récence: {recency_splits}")
    logger.info(f"Splits fréquence: {frequency_splits}")
    logger.info(f"Splits monétaire: {monetary_splits}")
    
    # Création des bucketizers
    recency_bucketizer = Bucketizer(
        splits=recency_splits, 
        inputCol="recency", 
        outputCol="recency_score",
        handleInvalid="keep"
    )
    
    frequency_bucketizer = Bucketizer(
        splits=frequency_splits, 
        inputCol="frequency", 
        outputCol="frequency_score",
        handleInvalid="keep"
    )
    
    monetary_bucketizer = Bucketizer(
        splits=monetary_splits, 
        inputCol="monetary", 
        outputCol="monetary_score",
        handleInvalid="keep"
    )
    
    # Appliquer les bucketizers
    rfm_df = recency_bucketizer.transform(rfm_df)
    rfm_df = frequency_bucketizer.transform(rfm_df)
    rfm_df = monetary_bucketizer.transform(rfm_df)
    
    # Inverser le score de récence et normaliser tous les scores
    rfm_df = rfm_df.withColumn("recency_score", 5.0 - col("recency_score"))
    rfm_df = rfm_df.withColumn("recency_score", 
                              when(col("recency_score") < 1, 1)
                              .when(col("recency_score") > 5, 5)
                              .otherwise(col("recency_score")))
    
    # Normaliser frequency et monetary scores
    for score_col in ["frequency_score", "monetary_score"]:
        rfm_df = rfm_df.withColumn(score_col, col(score_col) + 1)
        rfm_df = rfm_df.withColumn(score_col, 
                                  when(col(score_col) > 5, 5)
                                  .otherwise(col(score_col)))
    
    # Calculer le statut actif
    rfm_df = rfm_df.withColumn(
        "is_active",
        when(col("frequency") > 0, 1).otherwise(0)
    )
    
    # Calcul du score RFM global
    rfm_df = rfm_df.withColumn(
        "rfm_score", 
        col("recency_score") * 100 + col("frequency_score") * 10 + col("monetary_score")
    )
    
    # Segmentation RFM robuste
    rfm_df = rfm_df.withColumn(
        "rfm_segment",
        when(col("is_active") == 0, "Inactif")
        .when((col("recency_score") >= 4) & (col("frequency_score") >= 4) & (col("monetary_score") >= 4), "Champions")
        .when((col("recency_score") >= 4) & (col("frequency_score") >= 4), "Loyal Customers")
        .when((col("recency_score") >= 3) & (col("monetary_score") >= 4), "Big Spenders")
        .when((col("recency_score") >= 3) & (col("frequency_score") >= 3), "Potential Loyalists")
        .when((col("recency_score") <= 2) & (col("frequency_score") >= 3), "At Risk")
        .when((col("recency_score") <= 2) & (col("frequency_score") <= 2), "Hibernating")
        .when((col("recency_score") >= 4) & (col("frequency_score") <= 2), "New Customers")
        .when((col("recency_score") >= 3) & (col("frequency_score") <= 2), "Need Attention")
        .otherwise("Others")
    )
    
    # Afficher la distribution des segments - utilisation collect() pour éviter les erreurs UDF
    segment_distribution = rfm_df.groupBy("rfm_segment").count().orderBy(desc("count")).collect()
    
    logger.info("Distribution des segments RFM:")
    for row in segment_distribution:
        logger.info(f"{row['rfm_segment']}: {row['count']}")
    
    return rfm_df

In [113]:
def prepare_behavioral_clustering(user_df):
    """Prépare les données pour le clustering comportemental avec optimisations"""
    logger.info("Préparation des données pour clustering comportemental")
    
    # Sélection des features comportementales pertinentes
    behavior_features = [
        "nb_events", "nb_views", "nb_carts", "nb_purchases", "nb_removes",
        "avg_price_viewed", "avg_price_purchased", "nb_sessions",
        "conversion_rate", "cart_abandonment", "engagement_days"
    ]
    
    # Filtrer et mettre en cache
    clustering_df = user_df.filter(col("nb_events") >= 2).cache()
    logger.info(f"Utilisateurs avec au moins 2 événements: {clustering_df.count()}")
    
    # Remplacer les valeurs nulles par des zéros
    clustering_df = clustering_df.na.fill({
        feature: 0 for feature in behavior_features
    })
    
    # Assembler les features en vecteurs
    assembler = VectorAssembler(
        inputCols=behavior_features,
        outputCol="features_raw",
        handleInvalid="skip"
    )
    clustering_df = assembler.transform(clustering_df)
    
    # Standardiser les features
    scaler = StandardScaler(
        inputCol="features_raw", 
        outputCol="features",
        withStd=True, 
        withMean=True
    )
    
    # Pipeline pour le preprocessing
    preprocessing_pipeline = Pipeline(stages=[scaler])
    preprocessing_model = preprocessing_pipeline.fit(clustering_df)
    clustering_df = preprocessing_model.transform(clustering_df)
    
    return clustering_df, behavior_features

In [114]:
def train_kmeans_model(df, feature_col="features", k_values=range(2, 11)):
    """Entraîne et évalue plusieurs modèles K-means avec différentes valeurs de k"""
    logger.info("Entraînement des modèles K-means")
    
    # Liste pour stocker les résultats
    silhouette_scores = []
    models = {}
    
    # Évaluateur pour le clustering
    evaluator = ClusteringEvaluator(
        predictionCol="prediction", 
        featuresCol=feature_col,
        metricName="silhouette"
    )
    
    # Tester différentes valeurs de k
    for k in k_values:
        logger.info(f"Essai avec k={k}")
        
        try:
            # Créer et entraîner le modèle
            kmeans = KMeans(
                k=k, 
                seed=42, 
                featuresCol=feature_col,
                maxIter=20,
                tol=1e-4
            )
            model = kmeans.fit(df)
            
            # Faire des prédictions
            predictions = model.transform(df)
            
            # Évaluer le modèle
            silhouette = evaluator.evaluate(predictions)
            logger.info(f"Silhouette pour k={k}: {silhouette}")
            
            # Stocker les résultats
            silhouette_scores.append(silhouette)
            models[k] = model
            
        except Exception as e:
            logger.error(f"Erreur avec k={k}: {str(e)}")
            silhouette_scores.append(-1)  # Score invalide
            models[k] = None
    
    # Trouver la meilleure valeur de k en utilisant numpy
    silhouette_array = np.array(silhouette_scores)
    valid_indices = silhouette_array > -1  # Filtrer les scores invalides
    
    if not np.any(valid_indices):
        raise ValueError("Aucun modèle valide trouvé")
    
    # Trouver l'indice du meilleur score parmi les valides
    best_idx = np.argmax(silhouette_array[valid_indices])
    # Convertir l'indice local (parmi les valides) en indice global
    global_indices = np.where(valid_indices)[0]
    global_best_idx = global_indices[best_idx]
    
    best_k = list(k_values)[global_best_idx]
    best_score = silhouette_scores[global_best_idx]
    best_model = models[best_k]
    
    logger.info(f"Meilleur modèle: k={best_k} avec silhouette={best_score}")
    
    # Générer les prédictions avec le meilleur modèle
    results = best_model.transform(df)
    
    # Afficher la distribution des clusters
    logger.info("Distribution des clusters:")
    results.groupBy("prediction").count().orderBy("prediction").show()
    
    # Ajout de visualisation des scores silhouette
    try:
        plt.figure(figsize=(10, 6))
        plt.plot(list(k_values), silhouette_scores, 'bo-', linewidth=2, markersize=8)
        plt.xlabel('Nombre de clusters', fontsize=12)
        plt.ylabel('Score Silhouette', fontsize=12)
        plt.title('Optimisation du nombre de clusters - Score Silhouette', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Créer le timestamp pour le nom de fichier
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        plt.savefig(f"{RESULTS_DIR}/silhouette_scores_{timestamp_str}.png", dpi=300, bbox_inches='tight')
        plt.close()
        logger.info(f"Graphique des scores silhouette sauvegardé")
    except Exception as e:
        logger.warning(f"Erreur lors de la sauvegarde du graphique: {str(e)}")
    
    return best_model, results, best_k, silhouette_scores    
def analyze_clusters(df, cluster_col="prediction", feature_cols=None):
    """Analyse les caractéristiques des clusters avec gestion robuste"""
    logger.info("Analyse des caractéristiques des clusters")
    
    # Calculer les moyennes par cluster
    agg_exprs = [count("*").alias("cluster_size")]
    for col_name in feature_cols:
        agg_exprs.append(avg(col(col_name)).alias(f"avg_{col_name}"))

    cluster_stats = df.groupBy(cluster_col).agg(*agg_exprs).orderBy(cluster_col).collect()
    
    # Afficher les statistiques par cluster via logging
    logger.info("Statistiques par cluster:")
    for row in cluster_stats:
        logger.info(f"Cluster {row[cluster_col]}: {dict(row.asDict())}")
    
    return cluster_stats

In [115]:
def analyze_clusters(df, cluster_col="prediction", feature_cols=None):
    """Analyse les caractéristiques des clusters avec gestion robuste"""
    logger.info("Analyse des caractéristiques des clusters")
    
    # Calculer les moyennes par cluster
    agg_exprs = [count("*").alias("cluster_size")]
    for col_name in feature_cols:
        agg_exprs.append(avg(col(col_name)).alias(f"avg_{col_name}"))

    cluster_stats = df.groupBy(cluster_col).agg(*agg_exprs).orderBy(cluster_col).collect()
    
    # Afficher les statistiques par cluster via logging
    logger.info("Statistiques par cluster:")
    for row in cluster_stats:
        logger.info(f"Cluster {row[cluster_col]}: {dict(row.asDict())}")
    
    return cluster_stats
def combine_segmentations(cluster_df, rfm_df):
    """Combine les segmentations RFM et clustering comportemental - VERSION CORRIGÉE SANS UDF"""
    logger.info("Combinaison des segmentations RFM et clustering")
    
    # Debug : Vérifier les données d'entrée
    logger.info(f"Cluster DF count: {cluster_df.count()}")
    logger.info(f"RFM DF count: {rfm_df.count()}")
    
    # Jointure sécurisée avec repartitioning
    cluster_df = cluster_df.repartition(20, "user_id")
    rfm_df = rfm_df.repartition(20, "user_id").select("user_id", "rfm_segment", "rfm_score")
    
    combined_df = cluster_df.join(rfm_df, on="user_id", how="inner")
    
    # Renommer les colonnes pour plus de clarté
    combined_df = combined_df.withColumnRenamed("prediction", "behavior_cluster")
    
    # SOLUTION: Remplacer l'UDF par une expression CASE WHEN native Spark
    # Créer les étiquettes des clusters avec une expression conditionnelle
    combined_df = combined_df.withColumn(
        "behavior_segment",
        when(col("behavior_cluster") == 0, "Explorateurs Occasionnels")
        .when(col("behavior_cluster") == 1, "Acheteurs Fidèles")
        .when(col("behavior_cluster") == 2, "Visiteurs Fréquents")
        .when(col("behavior_cluster") == 3, "Acheteurs à Fort Panier")
        .when(col("behavior_cluster") == 4, "Visiteurs Uniques")
        .when(col("behavior_cluster") == 5, "Convertisseurs Efficaces")
        .when(col("behavior_cluster") == 6, "Indécis (Abandon Panier)")
        .when(col("behavior_cluster") == 7, "Browsers Passifs")
        .when(col("behavior_cluster") == 8, "Acheteurs Impulsifs")
        .otherwise("Segment Non Défini")
    )
    
    # Afficher les distributions via collect() pour éviter les erreurs
    logger.info("Distribution des segments comportementaux:")
    behavior_distribution = combined_df.groupBy("behavior_segment").count().orderBy(desc("count")).collect()
    for row in behavior_distribution:
        logger.info(f"{row['behavior_segment']}: {row['count']}")
    
    # Analyser l'affinité entre segments
    affinity_results = combined_df.groupBy("behavior_segment", "rfm_segment").count().orderBy(desc("count")).collect()
    logger.info("Top 10 affinités entre segments RFM et comportementaux:")
    for i, row in enumerate(affinity_results[:10]):
        logger.info(f"{i+1}. {row['behavior_segment']} + {row['rfm_segment']}: {row['count']}")
    
    return combined_df


In [116]:
def main():
    # Initialisation de la session Spark
    spark = create_spark_session()
    spark.sparkContext.setLogLevel("WARN")  # Réduire les logs Spark
    logger.info("Session Spark initialisée")
    
    try:
        # Trouver les fichiers les plus récents
        latest_cleaned = find_latest_file(DATA_DIR, "cleaned_data")
        latest_user_behavior = find_latest_file(DATA_DIR, "user_behavior")
        latest_recommendation = find_latest_file(DATA_DIR, "recommendation_data")
        latest_product = find_latest_file(DATA_DIR, "product_data")
        latest_time_series = find_latest_file(DATA_DIR, "time_series_data")
        
        # Vérifier que les fichiers existent
        if not latest_user_behavior:
            raise FileNotFoundError("Fichier user_behavior non trouvé")
        
        # Chargement des données principales
        logger.info("Chargement des données prétraitées")
        
        user_behavior_df = spark.read.parquet(os.path.join(DATA_DIR, latest_user_behavior))
        logger.info(f"Comportements utilisateurs chargés: {user_behavior_df.count()} lignes")
        
        # Exécuter la segmentation RFM
        rfm_segmentation = prepare_rfm_segmentation(user_behavior_df)
        
        # Exécuter la préparation pour le clustering
        behavior_clustering_df, behavior_features = prepare_behavioral_clustering(user_behavior_df)
        
        # Entraîner le modèle K-means
        kmeans_model, cluster_results, best_k, silhouette_scores = train_kmeans_model(
            behavior_clustering_df, feature_col="features", k_values=range(2, 8)
        )
        
        # Analyser les clusters obtenus
        cluster_stats = analyze_clusters(
            cluster_results.select("user_id", "prediction", *behavior_features),
            cluster_col="prediction", 
            feature_cols=behavior_features
        )
        
        # Sauvegarder le modèle
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"{MODELS_DIR}/kmeans_behavioral_{best_k}_clusters_{timestamp_str}"
        kmeans_model.save(model_path)
        logger.info(f"Modèle K-means sauvegardé: {model_path}")
        
        # Combiner les segmentations
        user_clusters = cluster_results.select("user_id", "prediction")
        combined_segments = combine_segmentations(user_clusters, rfm_segmentation)
        
        # Sauvegarder les segmentations combinées
        combined_output_path = f"{RESULTS_DIR}/combined_segmentation_{timestamp_str}.parquet"
        combined_segments.write.mode("overwrite").format("parquet").save(combined_output_path)
        logger.info(f"Segmentations combinées sauvegardées: {combined_output_path}")
        
        # Créer un dataframe de profils utilisateurs pour les recommandations
        user_profiles = combined_segments.select(
            "user_id", "behavior_cluster", "behavior_segment", "rfm_segment", "rfm_score"
        )
        
        # Sauvegarder les profils utilisateurs
        profiles_output_path = f"{RESULTS_DIR}/user_profiles_{timestamp_str}.parquet"
        user_profiles.write.mode("overwrite").format("parquet").save(profiles_output_path)
        logger.info(f"Profils utilisateurs sauvegardés: {profiles_output_path}")
        
        # Générer un graphique des scores silhouette (déjà fait dans train_kmeans_model, mais on peut le refaire ici si nécessaire)
        try:
            plt.figure(figsize=(10, 6))
            plt.plot(range(2, 8), silhouette_scores, 'bo-', linewidth=2, markersize=8)
            plt.xlabel('Nombre de clusters', fontsize=12)
            plt.ylabel('Score Silhouette', fontsize=12)
            plt.title('Optimisation du nombre de clusters - Score Silhouette', fontsize=14)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/silhouette_scores_final_{timestamp_str}.png", dpi=300, bbox_inches='tight')
            plt.close()
            logger.info(f"Graphique final des scores silhouette sauvegardé")
        except Exception as e:
            logger.warning(f"Erreur lors de la sauvegarde du graphique final: {str(e)}")
        
        # Afficher un résumé final
        logger.info("=== RÉSUMÉ DE LA SEGMENTATION ===")
        logger.info(f"Nombre d'utilisateurs total: {user_behavior_df.count()}")
        logger.info(f"Utilisateurs pour clustering: {behavior_clustering_df.count()}")
        logger.info(f"Nombre optimal de clusters: {best_k}")
        
        # Calculer le meilleur score silhouette de manière sécurisée
        valid_scores = [score for score in silhouette_scores if score > -1]
        best_silhouette = np.max(valid_scores) if valid_scores else 0
        logger.info(f"Score silhouette optimal: {best_silhouette:.4f}")
        logger.info(f"Utilisateurs segmentés: {combined_segments.count()}")
        
        # Afficher la distribution finale des segments
        logger.info("Distribution finale des segments RFM:")
        rfm_final_distribution = combined_segments.groupBy("rfm_segment").count().orderBy(desc("count")).collect()
        for row in rfm_final_distribution[:10]:
            logger.info(f"{row['rfm_segment']}: {row['count']}")
        
        logger.info("Distribution finale des segments comportementaux:")
        behavior_final_distribution = combined_segments.groupBy("behavior_segment").count().orderBy(desc("count")).collect()
        for row in behavior_final_distribution[:10]:
            logger.info(f"{row['behavior_segment']}: {row['count']}")
        
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        raise
    finally:
        spark.stop()
        logger.info("Session Spark fermée")

if __name__ == "__main__":
    main()

2025-05-18 23:59:03,008 - INFO - Session Spark initialisée
2025-05-18 23:59:03,012 - INFO - Chargement des données prétraitées
2025-05-18 23:59:03,427 - INFO - Comportements utilisateurs chargés: 163024 lignes
2025-05-18 23:59:03,429 - INFO - Préparation des données pour segmentation RFM
2025-05-18 23:59:05,229 - INFO - Splits récence: [0, 10, 20, 30, inf]
2025-05-18 23:59:05,231 - INFO - Splits fréquence: [0, 0.03819069584846403, 0.07638139169692806, 0.1145720875453921, inf]
2025-05-18 23:59:05,232 - INFO - Splits monétaire: [0, 16.671183476052605, 33.34236695210521, 50.013550428157814, inf]
2025-05-18 23:59:06,403 - INFO - Distribution des segments RFM:
2025-05-18 23:59:06,404 - INFO - Inactif: 150572
2025-05-18 23:59:06,406 - INFO - At Risk: 12452
2025-05-18 23:59:06,407 - INFO - Préparation des données pour clustering comportemental
2025-05-18 23:59:07,262 - INFO - Utilisateurs avec au moins 2 événements: 117529
2025-05-18 23:59:07,953 - INFO - Entraînement des modèles K-means
2025

+----------+------+
|prediction| count|
+----------+------+
|         0|107723|
|         1|  9806|
+----------+------+



2025-05-18 23:59:32,198 - INFO - Graphique des scores silhouette sauvegardé
2025-05-18 23:59:32,231 - INFO - Analyse des caractéristiques des clusters
2025-05-18 23:59:33,617 - INFO - Statistiques par cluster:
2025-05-18 23:59:33,618 - INFO - Cluster 0: {'prediction': 0, 'cluster_size': 107723, 'avg_nb_events': 7.845093434085571, 'avg_nb_views': 7.777057824234379, 'avg_nb_carts': 0.04321268438495029, 'avg_nb_purchases': 0.02482292546624212, 'avg_nb_removes': 0.0, 'avg_avg_price_viewed': 320.8869577016758, 'avg_avg_price_purchased': 2.1138300084475916, 'avg_nb_sessions': 1.501072194424589, 'avg_conversion_rate': 0.002730616324765064, 'avg_cart_abandonment': 0.027586185556164114, 'avg_engagement_days': 1.0}
2025-05-18 23:59:33,620 - INFO - Cluster 1: {'prediction': 1, 'cluster_size': 9806, 'avg_nb_events': 11.157250662859473, 'avg_nb_views': 8.6962064042423, 'avg_nb_carts': 1.018050173363247, 'avg_nb_purchases': 1.4429940852539263, 'avg_nb_removes': 0.0, 'avg_avg_price_viewed': 372.51225

In [117]:
def prepare_recommendation_data(recom_df):
    """Prépare les données pour le système de recommandation"""
    logger.info("Préparation des données pour le système de recommandation")
    
    # Indexer les utilisateurs et produits pour ALS
    user_indexer = StringIndexer(
        inputCol="user_id", 
        outputCol="user_idx",
        handleInvalid="skip"
    )
    
    product_indexer = StringIndexer(
        inputCol="product_id", 
        outputCol="product_idx",
        handleInvalid="skip"
    )
    
    # Créer le pipeline de préparation
    pipeline = Pipeline(stages=[user_indexer, product_indexer])
    pipeline_model = pipeline.fit(recom_df)
    als_df = pipeline_model.transform(recom_df)
    
    # Afficher un aperçu des données préparées
    logger.info("Aperçu des données préparées pour ALS:")
    als_df.select("user_id", "user_idx", "product_id", "product_idx", 
                 "event_type", "interaction_score").show(5)
    
    # Calculer quelques statistiques utiles
    unique_users = als_df.select("user_id").distinct().count()
    unique_products = als_df.select("product_id").distinct().count()
    total_interactions = als_df.count()
    
    logger.info(f"Statistiques du dataset de recommandation:")
    logger.info(f"Utilisateurs uniques: {unique_users}")
    logger.info(f"Produits uniques: {unique_products}")
    logger.info(f"Interactions totales: {total_interactions}")
    logger.info(f"Densité: {total_interactions / (unique_users * unique_products) * 100:.6f}%")
    
    # Diviser les données en ensembles d'entraînement et de test
    train_df, test_df = als_df.randomSplit([0.8, 0.2], seed=42)
    
    logger.info(f"Ensemble d'entraînement: {train_df.count()} lignes")
    logger.info(f"Ensemble de test: {test_df.count()} lignes")
    
    return train_df, test_df, pipeline_model

# Préparer les données pour les recommandations
als_train_df, als_test_df, als_pipeline = prepare_recommendation_data(recommendation_df)

2025-05-18 23:59:47,584 - INFO - Préparation des données pour le système de recommandation


AssertionError: 

In [ ]:
def train_als_model(train_df, test_df):
    """Entraîne et évalue le modèle ALS"""
    logger.info("Entraînement du modèle ALS")
    
    # Hyperparamètres à tester
    als_models = {}
    ranks = [10, 20, 30]
    reg_params = [0.01, 0.1, 1.0]
    
    best_model = None
    best_rmse = float('inf')
    
    for rank in ranks:
        for reg_param in reg_params:
            logger.info(f"Essai avec rank={rank}, regParam={reg_param}")
            
            als = ALS(
                rank=rank,
                maxIter=15,
                regParam=reg_param,
                userCol="user_idx",
                itemCol="product_idx",
                ratingCol="interaction_score",
                coldStartStrategy="drop",
                nonnegative=True,
                implicitPrefs=True
            )
            
            model = als.fit(train_df)
            predictions = model.transform(test_df)
            
            # Évaluation
            evaluator = RegressionEvaluator(
                metricName="rmse",
                labelCol="interaction_score",
                predictionCol="prediction"
            )
            rmse = evaluator.evaluate(predictions)
            
            logger.info(f"RMSE pour rank={rank}, regParam={reg_param}: {rmse}")
            
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
    
    logger.info(f"Meilleur modèle - RMSE: {best_rmse}")
    
    # Sauvegarde du modèle
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"{MODELS_DIR}/als_model_{timestamp_str}"
    best_model.save(model_path)
    logger.info(f"Modèle ALS sauvegardé: {model_path}")
    
    return best_model

# Exécuter l'entraînement ALS
als_model = train_als_model(als_train_df, als_test_df)

2025-05-18 17:58:57,717 - INFO - Entraînement du modèle ALS
2025-05-18 17:58:57,719 - INFO - Essai avec rank=10, regParam=0.01
2025-05-18 17:59:50,935 - INFO - RMSE pour rank=10, regParam=0.01: 1.687387446788512
2025-05-18 17:59:50,936 - INFO - Essai avec rank=10, regParam=0.1
2025-05-18 18:00:34,282 - INFO - RMSE pour rank=10, regParam=0.1: 1.6891501157680544
2025-05-18 18:00:34,283 - INFO - Essai avec rank=10, regParam=1.0
2025-05-18 18:01:21,609 - INFO - RMSE pour rank=10, regParam=1.0: 1.730507037766797
2025-05-18 18:01:21,611 - INFO - Essai avec rank=20, regParam=0.01
2025-05-18 18:02:13,881 - INFO - RMSE pour rank=20, regParam=0.01: 1.667358695414784
2025-05-18 18:02:13,882 - INFO - Essai avec rank=20, regParam=0.1
2025-05-18 18:03:04,332 - INFO - RMSE pour rank=20, regParam=0.1: 1.6690093297310653
2025-05-18 18:03:04,333 - INFO - Essai avec rank=20, regParam=1.0
2025-05-18 18:03:59,786 - INFO - RMSE pour rank=20, regParam=1.0: 1.7204041942327433
2025-05-18 18:03:59,787 - INFO - 

In [ ]:
def prepare_content_based_features(product_df):
    """Prépare les caractéristiques produits pour le content-based filtering"""
    logger.info("Préparation des caractéristiques produits")
    
    # Combiner les métadonnées
    product_features = product_df.withColumn(
        "product_features",
        concat_ws(" ", 
            col("category_code"), 
            col("brand"), 
            col("price").cast("string")
        )
    )
    
    # TF-IDF
    tokenizer = Tokenizer(inputCol="product_features", outputCol="tokens")
    hashing_tf = HashingTF(inputCol="tokens", outputCol="raw_features", numFeatures=1000)
    idf = IDF(inputCol="raw_features", outputCol="features")
    
    pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf])
    model = pipeline.fit(product_features)
    product_features = model.transform(product_features)
    
    return product_features

# Utilisation
product_features = prepare_content_based_features(product_df)

2025-05-18 16:40:42,419 - INFO - Préparation des caractéristiques produits


In [118]:
def get_user_preferences(user_id, als_model):
    """Récupère le vecteur latent sous forme de liste"""
    user_factors = als_model.userFactors.filter(col("id") == user_id)
    if user_factors.count() == 0:
        return None
    return user_factors.first().features.tolist()

def cosine_similarity(v1, v2):
    """Calcule la similarité cosinus entre deux vecteurs"""
    return float(v1.dot(v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

def combine_recommendations(als_recs, content_recs, alpha=0.7):
    """Combine les recommandations avec pondération"""
    combined = als_recs.union(content_recs).groupBy("product_id").agg(
        (alpha * max("rating")).alias("als_score"),
        ((1 - alpha) * max("similarity")).alias("content_score"),
        (alpha * max("rating") + (1 - alpha) * max("similarity")).alias("combined_score")
    ).orderBy(desc("combined_score"))
    
    return combined

def hybrid_recommendations(user_id, als_model, product_features, num_recs=10):
    """Combine les recommandations collaboratives et basées sur le contenu"""
    # Conversion préalable du user_id en DataFrame
    user_df = spark.createDataFrame([(user_id,)], ["user_id"])
    
    # Récupération des recommandations ALS (avec gestion de la sérialisation)
    als_recs = als_model.recommendForUserSubset(user_df, num_recs)
    
    # Extraction du vecteur utilisateur sous forme de liste Python
    user_vector = get_user_preferences(user_id, als_model)
    if user_vector is None:
        return als_recs
    
    user_array = user_vector.tolist()  # Conversion en liste sérialisable
    
    # Définition de l'UDF avec capture de la liste Python
    def similarity_udf_wrapper(features):
        user_vec = np.array(user_array)
        product_vec = np.array(features.toArray())
        dot_product = np.dot(user_vec, product_vec)
        norm_product = np.linalg.norm(product_vec)
        return float(dot_product / (np.linalg.norm(user_vec) * norm_product))
    
    similarity_udf = udf(similarity_udf_wrapper, FloatType())
    
    # Calcul des similarités
    content_recs = product_features.withColumn(
        "similarity", similarity_udf(col("features"))
    ).select("product_id", "similarity").orderBy(desc("similarity")).limit(num_recs)
    
    # Combinaison des résultats
    als_df = als_recs.selectExpr(
        "product_id", 
        "explode(recommendations).rating"
    )
    
    return combine_recommendations(als_df, content_recs)    

### 2. Implémentation complète de enhanced_rfm_segmentation ###
def enhanced_rfm_segmentation(user_df):
    """Amélioration de la segmentation RFM avec validation"""
    # Calcul des percentiles dynamiques
    window_r = Window.orderBy(desc("recency"))
    window_fm = Window.orderBy(col("frequency"), col("monetary"))
    
    rfm_df = user_df.withColumn(
        "r_percentile", percent_rank().over(window_r)
    ).withColumn(
        "f_percentile", percent_rank().over(window_fm)
    ).withColumn(
        "m_percentile", percent_rank().over(window_fm)
    )
    
    # Calcul du score pondéré
    rfm_df = rfm_df.withColumn(
        "rfm_score",
        (col("r_percentile") * 0.4 + 
         col("f_percentile") * 0.3 + 
         col("m_percentile") * 0.3)
    )
    
    # Clustering avec validation
    assembler = VectorAssembler(inputCols=["rfm_score"], outputCol="features")
    kmeans = KMeans(k=5, seed=42)
    pipeline = Pipeline(stages=[assembler, kmeans])
    model = pipeline.fit(rfm_df)
    
    # Évaluation
    predictions = model.transform(rfm_df)
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    logger.info(f"Silhouette Score pour RFM amélioré: {silhouette}")
    
    return predictions

### 3. Intégration dans le flux principal ###
# Après l'entraînement du modèle ALS et la segmentation RFM initiale

# Segmentation RFM améliorée
enhanced_rfm = enhanced_rfm_segmentation(user_behavior_df)
enhanced_rfm.select("user_id", "rfm_score", "prediction").show(5)

# Exemple d'utilisation des recommandations hybrides
sample_user = str(user_behavior_df.first().user_id)

hybrid_recs = hybrid_recommendations(
    user_id=sample_user,
    als_model=als_model,
    product_features=product_features,
    num_recs=10
)

logger.info("Recommandations hybrides pour l'utilisateur %s:", sample_user)
hybrid_recs.show(10)

# Sauvegarde des résultats
hybrid_recs.write.mode("overwrite").parquet(f"{RESULTS_DIR}/hybrid_recommendations")
enhanced_rfm.write.mode("overwrite").parquet(f"{RESULTS_DIR}/enhanced_rfm_segments")

AssertionError: 